In [2]:
import tensorflow.compat.v1 as tf
import keras
from keras import applications
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import os

In [3]:
!ls

'Baseline_CNN Notebook.ipynb'   TestData.ipynb	     session.py
'MNIST_CNN Notebook.ipynb'      data
 Max_CCT_CNN.ipynb	        resnet_model.ipynb


In [4]:
labels = np.load('data/labels.npy')
animals = np.load('data/label_to_animal.npz')
for i in animals:
    print(i)
animals = animals['arr_0']
animals.shape

arr_0


(16,)

In [5]:
images = np.load('data/images.npy')
images.shape

(1000, 500, 1000, 3)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3)

In [7]:
base_model = applications.resnet50.ResNet50(weights="imagenet", include_top=False, input_shape= (500, 1000, 3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 2s 0us/step


In [8]:
base_model.save("base_model.h5")

In [28]:
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
predictions = keras.layers.Dense(16, activation= 'softmax')(x)
model = keras.models.Model(inputs = base_model.input, outputs = predictions)

In [31]:
from keras.optimizers import Adam
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
#changed loss to binary crossentropy from categorical_crossentropy to see what would happen

In [32]:
model.fit(X_train, y_train, epochs = 10, batch_size = 32,
          validation_data=(X_test, y_test))

Train on 700 samples, validate on 300 samples
Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[32,125,250,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node res2a_branch1_1/BiasAdd-0-6-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_180876]

Function call stack:
keras_scratch_graph


In [ ]:
print('\n# Evaluate on test data')
results = model.evaluate(X_test, y_test, batch_size=8)
print('test loss, test acc:', results)

In [ ]:
model.save("model.h5")